In [121]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle

In [122]:
png_files = []
#folder_path = r"C:\Users\sacha\OneDrive\Desktop\python\cs771\assn2\train"
for i in range(0, 2000):
    file_path = "C:\\Users\\sacha\\OneDrive\\Desktop\\python\\cs771\\assn2\\train\\{}.png".format(i)
    png_files.append(file_path)


In [123]:
png_files[4]

'C:\\Users\\sacha\\OneDrive\\Desktop\\python\\cs771\\assn2\\train\\4.png'

In [126]:
cropped_train = []

def cropper(i):
    image = Image.open(png_files[i])
    width, height = image.size

    crop_region = (width - 140, 10, width - 50, height)
    ci = image.crop(crop_region)
    ci = np.asarray(ci)
    
    _,img = cv2.threshold(ci,165,255,cv2.THRESH_BINARY)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,img = cv2.threshold(img,254,255,cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    img_erode = cv2.dilate(img, kernel)
    cropped_train.append(img_erode)

In [127]:
for i in range(2000):
    cropper(i)
x_train = np.array(cropped_train)
file = open(r"C:\Users\sacha\OneDrive\Desktop\python\cs771\assn2\train\labels.txt", "r")
temp = file.read().splitlines()
y_train = np.array(temp)

In [128]:
Y_train =[]
for i in range(2000):
    if y_train[i] == 'ODD':
        Y_train.append(1)
    else:
        Y_train.append(0)
Y_train = np.array(Y_train)

In [129]:
X_trn, x_tst,Y_trn, y_tst = train_test_split(x_train,Y_train,test_size=0.25)

In [130]:
X_trn = X_trn / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(2, kernel_size=(3, 3), activation='relu', input_shape=(90, 90, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(4, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_trn, Y_trn, epochs=30, batch_size=32)
# model.save_weights('model.h5')
weights_file = "final.pkl"
with open(weights_file, "wb") as f:
    pickle.dump(model, f)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 88, 88, 2)         20        
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 44, 44, 2)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 42, 42, 4)         76        
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 21, 21, 4)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 19, 19, 8)         296       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 9, 9, 8)          

In [131]:
predictions = model.predict(x_tst)
count = 0
for i in range(500):
    if predictions[i] == y_tst[i]:
        count = count + 1
count/5.0

16/16 [==============================] - 0s 7ms/step


100.0